In [2]:
import pandas as pd
import numpy as np

SOLVER_MILO = "highs"
SOLVER_MINLO = "ipopt"

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["coin", "highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

AMPL Development Version 20240404 (MSVC 19.38.33135.0, 64-bit)
Demo license with maintenance expiring 20260131.
Using license file "c:\Users\thuduong\Anaconda3\envs\optima\Lib\site-packages\ampl_module_base\bin\ampl.lic".



In [3]:
# # TEST 1
# stocks = {
#     "A": {"width": 1219, "weight": 3657,"unit":3 },
#     "B": {"width": 1219, "weight": 400,"unit": 0.33},
#     "C": {"width": 1108, "weight": 4432, "unit":4},
#     "D": {"width": 1219, "weight": 4876, "unit":4},
# }

# finish = {
#     "XXL": {"width": 220, "need_cut": 4247 , "fc1":22574, "upper_cut":4247},
#     "L": {"width": 145, "need_cut": 1895, "fc1": 4559, "upper_cut":4559},
#     "XL": {"width": 150, "need_cut": 788, "fc1": 4618, "upper_cut":4618},
#     "M": {"width": 70, "need_cut": 515, "fc1": 669, "upper_cut":669},
#     "S": {"width": 52, "need_cut": 400, "fc1": 0, "upper_cut":669},
# }

In [4]:
# # TEST 1
# stocks = {
#     "HTV0828/23-D1": {"width": 1108, "weight": 4266 },
#     "HTV0895/23": {"width": 1219, "weight": 5350},
#     "HTV1363/23": {"width": 1108, "weight": 4000},
# }

# finish = {
#     "CSC JSH590R-PO 2.60X225XC": {"width": 225, "need_cut": 800 , "fc1":6522},
#     "POSCO JSH270C-PO 2.60X106XC": {"width": 106, "need_cut": 1414, "fc1": 10417},
#     "POSCO JSH270C-PO 2.60X120XC": {"width": 120, "need_cut": 50, "fc1": 269},
#     "POSCO JSH270C-PO 2.60X220XC": {"width": 220, "need_cut": 4247, "fc1": 22574},
#     "POSCO JSH270C-PO 2.60X150XC": {"width": 150, "need_cut": 788, "fc1": 4619},
#     "POSCO JSH270C-PO 2.60X127XC": {"width": 127, "need_cut": 100, "fc1": 0},
#     "POSCO JSH270C-PO 2.60X145XC": {"width": 145, "need_cut": 1895, "fc1": 4560},
# }

In [44]:
# TEST 1
stocks = {
    "S1": {"width": 1108, "weight": 4266 },
    "S2": {"width": 1219, "weight": 5350},
    "S3": {"width": 1219, "weight": 4000},
    # "S4": {"width": 1219, "weight": 400},
}

finish = {
    "F1": {"width": 225, "need_cut": 800 , "fc1":6522},
    "F2": {"width": 106, "need_cut": 1414, "fc1": 10417},
    "F3": {"width": 220, "need_cut": 4247, "fc1": 22574},
    "F4": {"width": 150, "need_cut": 788, "fc1": 4619},
    "F5": {"width": 127, "need_cut": 300, "fc1": 0},
    "F6": {"width": 145, "need_cut": 1895, "fc1": 4560},
    "F7": {"width": 52, "need_cut": 100, "fc1": 104},
    "F8": {"width": 70, "need_cut": 515, "fc1": 669},
    "F9": {"width": 72, "need_cut": 200, "fc1": 912}, 
}

In [45]:
def make_patterns_by_weight_width(stocks, finish, bound=None):
    """
    Generates patterns of feasible cuts from stock lengths to meet specified finish lengths.

    Parameters:
    stocks (dict): A dictionary where keys are stock identifiers and values are dictionaries
                   with key 'length' representing the length of each stock.

    finish (dict): A dictionary where keys are finish identifiers and values are dictionaries
                   with key 'length' representing the required finish lengths.

    Returns:
    patterns (list): A list of dictionaries, where each dictionary represents a pattern of cuts.
                   Each pattern dictionary contains 'stock' (the stock identifier) and 'cuts'
                   (a dictionary where keys are finish identifiers and the value is the number
                   of cuts from the stock for each finish).
                   

                   Naive pattern with maximum number of cuts that is closet to the required need_cut
                   and SUM(length) smaller Mother Coil length
    """
    if bound is None:
        bound = 0.3
    else:
        pass

    patterns = []
    for f in finish:
        feasible = False
        # finish[f]["upper_cut"] = finish[f]["need_cut"] + 

        for s in stocks:
            # max number of f that fit on s 
            num_cuts_by_width = int(stocks[s]["width"] / finish[f]["width"])
            # max number of f that satisfied the need cut
            upper_demand_finish = finish[f]["need_cut"] + bound * finish[f]["need_cut"]
            num_cuts_by_weight = round((upper_demand_finish*stocks[s]["width"] ) / (finish[f]["width"]*stocks[s]['weight']))
            # min of two max will satisfies both
            num_cuts = min(num_cuts_by_width, num_cuts_by_weight)

            # make pattern and add to list of patterns
            if num_cuts > 0:
                feasible = True
                cuts_dict = {key: 0 for key in finish.keys()}
                cuts_dict[f] = num_cuts
                patterns.append({"stock": s, "cuts": cuts_dict})
        if not feasible:
            print(f"No feasible pattern was found for Stock {s} and FG {f}")
            # return []

    return patterns

patterns = make_patterns_by_weight_width(stocks, finish,bound=None)
print("Set of naive pattern - which is [ x, x, max, x, x ]")
# display(patterns)

Set of naive pattern - which is [ x, x, max, x, x ]


A solution is attempted using a mixed-integer nonlinear optimization (MINLO) solver.

In [46]:
pattern_bilinear_prob = AMPL()
pattern_bilinear_prob.option["solver"] = SOLVER_MINLO
pattern_bilinear_prob.eval(
    """
        set S;
        set F;
        set P;

        ## PARAM
        # weight unit of stock - weight/width
        # param weight_perunit_s{P union S};
        param weight_perunit_s{S};
        
        # width stock
        param width_s{S};

        # width finished pieces
        param width_f{F};

        param demand_finish{F};
        param upper_demand_finish{F};
        param a{F, P};
        param ap_upper_bound{F};

        # pattern by finished good bounded by [0, upper bound]
        var ap{f in F} integer >= 0 <= ap_upper_bound[f];
        # choosing stock or not (0, 1)
        var bp{S} binary;

        minimize trim_loss:
          sum{s in S} bp[s] * width_s[s] - sum{f in F} ap[f] * width_f[f];

        subject to find_stocks:
          sum{s in S} bp[s] = 1;

        subject to min_margin:
          sum{f in F} ap[f] * width_f[f] + 8 <= sum{s in S} bp[s] * width_s[s];

        subject to max_margin:
          0.96 * sum{s in S} bp[s] * width_s[s] <= sum{f in F} ap[f] * width_f[f];

        subject to upper_demand {f in F}:
          width_f[f] * ap[f] * sum {s in S} bp[s] * weight_perunit_s[s] <= upper_demand_finish[f];
"""
)

In [52]:
def create_pattern_bilinear(stocks, finish, patterns):
    m = pattern_bilinear_prob
    m.eval("reset data;")
    m.set["S"] = list(stocks.keys())
    m.set["F"] = list(finish.keys())
    m.set["P"] = list(range(len(patterns))) # list number

    s = {p: patterns[p]["stock"] for p in range(len(patterns))} # stock allocated to naive patterns (0, n)

    # w = {p: stocks[s[p]]["weight"]/stocks[s[p]]["width"] for p in range(len(patterns))} # weight per unit according to each naive pattern
    wstocks = {s: stocks[s]["weight"]/stocks[s]["width"] for s in list(stocks.keys())} # stock weight per unit (unique)
    # w.update(wstocks)

    m.param["weight_perunit_s"] = wstocks
    a = { # list down (finish good name, pattern number code: number of cuts) naive patterns
        (f, p): patterns[p]["cuts"][f]
        for p in range(len(patterns))
        for f in finish.keys()
    }

    # over cut bound
    bound = 0.3

    m.param["a"] = a
    m.param["width_s"] = {s: stocks[s]["width"] for s in stocks.keys()}
    m.param["width_f"] = {f: finish[f]["width"] for f in finish.keys()}
    m.param["demand_finish"] = {f: finish[f]["need_cut"] for f in finish.keys()}
    m.param["upper_demand_finish"] = {f: finish[f]["need_cut"] + bound * finish[f]["need_cut"] for f in finish.keys()}

    ap_upper_bound = { #upper bound of possible cut over all kind of stocks
        f: max([patterns[i]['cuts'][f] for i,_ in enumerate(patterns)])
        for f in finish.keys()
    } # fix lai by weight va by width 


    m.param["ap_upper_bound"] = ap_upper_bound
    m.get_output("solve;")
    bp_values = dict(m.var["bp"].get_values())
    ap_values = dict(m.var["ap"].get_values())

    # Retrieve the patterns
    sorted_stocks = sorted([s for s in stocks.keys() if bp_values[s] > 0.5], reverse=True)
    try:
        new_pattern = {
            "stock": sorted_stocks[0],
            "stocks_width":stocks[sorted_stocks[0]]['width'],
            "cuts": {f: round(ap_values[f]) for f in finish.keys()},
            "trim_loss": stocks[sorted_stocks[0]]['width'] - sum([finish[f]['width'] * round(ap_values[f]) for f in finish.keys()])
        }
    except:
        print("cant find the optimal pattern")

    return bp_values, new_pattern

bp_values, new_pattern = create_pattern_bilinear(stocks, finish, patterns)

cant find the optimal pattern


UnboundLocalError: cannot access local variable 'new_pattern' where it is not associated with a value